# Decorators

A decorator is a function whose primary purpose is to wrap another function or class. The primary purpose of this wrapping is to transparently alter or enhance the behavior of the object being wrapped.

In [1]:
def f(n):
    return list(range(n))

In [2]:
# decorator function takes a function, modifies its behaviour and returns another function
def verbose(func):
    def tempfunc(*args, **kwargs):
        print('args: ', args)
        print('kwargs: ', kwargs)
        result = func(*args, **kwargs)
        return result
    return tempfunc

In [3]:
g = verbose(f)

In [4]:
f(10)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [5]:
g(10)

args:  (10,)
kwargs:  {}


[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [6]:
@verbose
def h(x, y):
    return "something"

In [7]:
h(1, y="anything")

args:  (1,)
kwargs:  {'y': 'anything'}


'something'

## Function decorators w/o arguments

In [8]:
def licensed(func):
    from datetime import datetime
    end_date = datetime(2018,1,1,0,0)
    def wrapper(*args, **kwargs):
        if datetime.now() < end_date:
            return func(*args, **kwargs)
        else:
            print(end_date.strftime("Your license has expired since %d %b %Y %H:%M"))
            return None
    return wrapper

```
@decorator_function
def f():
    return

f = decorator_function(f) # f is not the same f anymore
f(n)

```

In [9]:
@licensed
def hello(name):
    return 'hello ' + name

In [10]:
hello("World")

Your license has expired since 01 Jan 2018 00:00


In [11]:
import functools # use functools.wraps to inject function's properties

def licensed2(func):
    from datetime import datetime
    end_date = datetime(2015,11,13,12,00)
    
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        if datetime.now() < end_date:
            return func(*args, **kwargs)
        else:
            print(end_date.strftime("Your license has expired since %d %b %Y %H:%M"))
            return None
    return wrapper

In [12]:
@licensed2
def hello2(name):
    return 'hello ' + name

In [13]:
print(hello.__name__)
print(hello2.__name__)

wrapper
hello2


## Function decorators with arguments

In [14]:
import functools

def licensed(end_date):
    def decorator(func):      
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            from datetime import datetime
            if datetime.now() < end_date:
                return func(*args, **kwargs)
            else:
                print(end_date.strftime("Your license has expired since %d %b %Y %H:%M"))
                return None
        return wrapper
    return decorator

In [15]:
from datetime import datetime

@licensed(datetime(2015,11,13,12,0))
def hola(name):
    return 'hola ' + name

```
@decorator_function(arg1)
def f():
    return

real_decorator_function = decorator_function(arg1)
f = real_decorator_function(f) # modify f

f(n)

```

### A function decorator to create decorators (ultimate example)

In [17]:
def decorator(declared_decorator):
    """Create a decorator out of a function, which will be used as a wrapper."""
 
    @functools.wraps(declared_decorator)
    def final_decorator(func=None, **kwargs):
        # This will be exposed to the rest
        # of your application as a decorator
 
        def decorated(func):
            # This will be exposed to the rest
            # of your application as a decorated
            # function, regardless how it was called
            @functools.wraps(func)
            def wrapper(*a, **kw):
                # This is used when actually executing
                # the function that was decorated
                return declared_decorator(func, a, kw, **kwargs)
            return wrapper
 
        if func is None:
            # The decorator was called with arguments,
            # rather than a function to decorate
            return decorated
        else:
            # The decorator was called without arguments,
            # so the function should be decorated immediately
            return decorated(func)
    return final_decorator

In [18]:
@decorator
def suppress_errors(func, args, kwargs, log_func=None):
    try:
        return func(*args, **kwargs)
    except Exception as e:
        if log_func is not None:
             log_func(str(e))
        else:
            print('suppressed error: ', e)

In [19]:
@suppress_errors
def example():
    return variable_which_does_not_exist

In [20]:
@suppress_errors
def example2(a,b=2):
    print(a,b)
    return variable_which_does_not_exist

In [21]:
example()

suppressed error:  name 'variable_which_does_not_exist' is not defined


In [22]:
example2(1,b=3)

1 3
suppressed error:  name 'variable_which_does_not_exist' is not defined


## Class decorators without arguments
The mechanism for class decorators without arguments is quite simple. The expression is actually:
<br>
decorated_func = decorator_wo_arguments(func)

which means "__init__" method is fired when the decorator is implemented.

In [23]:
class memoize:
    def __init__(self,f):
        self.fn = f
        self.memo = {}
        
    def __call__(self,*args):
        if args not in self.memo:
            self.memo[args] = self.fn(*args)
        return self.memo[args]   

In [24]:
def fibo(n):
    if n==0:
        return 0
    if n==1:
        return 1
    else:
        return fibo(n-1) + fibo(n-2)

In [25]:
@memoize
def fiboM(n):
    if n==0:
        return 0
    if n==1:
        return 1
    else:
        return fibo(n-1) + fibo(n-2)

In [28]:
%timeit fibo(20)

2.8 ms ± 53.5 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [32]:
%timeit fiboM(20)

289 ns ± 5.21 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


```
@decorator_class 
def f():
    return
    
f = decorator_class.__init__(f) # f is not a function anymore but a callable object
f(n) --> f.__call__(arguments)

```

## Class decorators with arguments
The mechanism for class decorators with arguments is slightly different. Since it takes arguments, the expression becomes:
<br>
decorated_func = decorator_w_arguments(*args)(func)

So for this case, "__call__" method is fired when the decorator is implemented not "__init__" as in decorators with arguments

In [29]:
class delay:
    def __init__(self,t):
        self.t = t
        
    def __call__(self,f):
        print('inside __call__')
        print('this is printed once when decorator is created/implemented')
        def wrapper(*args):
            print('inside wrapper')
            print('this is printed whenever the decorated function is called')
            return f(*args)
        
        return wrapper

In [30]:
@delay(10)
def pw2(a):
    return a**2

inside __call__
this is printed once when decorator is created/implemented


In [31]:
pw2(5)

inside wrapper
this is printed whenever the decorated function is called


25

```
@decorator_class(arg1)
def f():
    return

decorator_instance = decorator_class.__init__(arg1) # decorator class is instantiated
f = decorator_instance.__call__(f) # decorator instance is a callable object
f(n) --> f.__call__(arguments)

```